In [34]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from datetime import datetime
from datetime import timedelta

df1 = pd.read_csv('walks.csv') # Walks
walks_df = df1.copy()
df2 = pd.read_csv('walkers.csv') # Walkers
walkers_df = df2.copy()
df3 = pd.merge(df1,df2, how = 'inner', left_on = 'walkerid', right_on = 'id')
df3copy = df3.copy()

In [35]:
g = df3.groupby('walkerid').agg({'walkid': ['count'],
                                 'checkintime' : ['min','max'],
                                 'signuptime' : [min],
                                 "district": pd.Series.nunique})
g.columns = ['totalWalks','firstWalk','lastWalk','signuptime','differentDistrictCount']
g.reset_index()

,walkerid,totalWalks,firstWalk,lastWalk,signuptime,differentDistrictCount
0,01b0fb51-4461-4638-95a5-8fc8c0ef1c9d,2,2020-03-10 11:15:00.000,2020-03-12 12:00:00.000,2020-03-09 14:52:09.482,1
1,01d9cdcd-c770-4d29-8ca5-dd0770962b67,3,2020-02-10 15:45:00.000,2020-03-15 05:00:00.000,2020-02-08 17:45:59.828,1
2,01fa4741-4138-472b-bdb6-d1f19fd2f44a,1,2019-07-17 17:00:00.000,2019-07-17 17:00:00.000,2019-07-01 14:50:38.745,1
3,03b85e74-989f-4cf2-993f-d8411de50e3a,4,2020-02-05 10:54:30.993,2020-02-07 13:45:00.000,2020-02-04 09:59:30.186,2
4,045cc0a5-afb1-4caa-9575-7faf6d3e1f69,18,2020-03-05 12:00:00.000,2020-04-15 08:30:00.000,2020-03-04 12:39:17.707,2
...,...,...,...,...,...,...
350,fc597c8e-65bc-48f6-b5d9-b372238cd94d,5,2020-03-29 18:00:00.000,2020-09-29 09:00:00.000,2020-03-15 17:49:50.346,2
351,fd1c862f-25a6-4051-a9b5-2776fb28345e,2,2019-08-31 17:00:00.000,2019-10-18 07:00:00.000,2019-08-26 09:24:47.883,1
352,fde176dc-7f6b-408f-8c3e-6835b651d5ba,23,2020-01-25 18:00:00.000,2020-08-22 15:30:00.000,2020-01-24 12:44:10.224,2
353,ff665506-7322-4fcf-adef-0fdc52a8bd71,171,2019-09-14 09:00:00.000,2021-02-25 15:45:00.000,2019-09-09 18:54:57.236,3


In [36]:

# averageWalk (19)
# Average walk amount per walker is 19.07
# Finding if a walker did more or less walks than the average, and labeling them accordingly
totalWalks_mean = g['totalWalks'].mean()
g.loc[(g['totalWalks'] > totalWalks_mean), ['averageWalk']] = 'MORE'
g.loc[(g['totalWalks'] < totalWalks_mean), ['averageWalk']] = 'LESS'

# lifetime
g['firstWalk'] =  pd.to_datetime(g['firstWalk']) # Converting columns 'firstwalk', 'lastWalk' and 'signuptime' into datetime
g['lastWalk'] =  pd.to_datetime(g['lastWalk']) 
g['signuptime'] =  pd.to_datetime(g['signuptime'])
g['lifetime'] = g['lastWalk'] - g['signuptime'] # Finding the difference between 'lastWalk' and 'signuptime'

# averageLifetime(102 days)
lifetime_mean = g['lifetime'].mean()
g.loc[(g['lifetime'] > lifetime_mean), ['averageLifetime']] = 'MORE'
g.loc[(g['lifetime'] < lifetime_mean), ['averageLifetime']] = 'LESS'

# firstWalkAfterSignup
g['firstWalkAfterSignup'] = g['firstWalk'] - g['signuptime'] # Finding the difference between 'firstWalk' and 'signuptime'

# firstWalkinWeek
g.loc[(g['firstWalk'] - g['signuptime']) < timedelta(days=7), ['firstWalkinWeek']] = 'YES' 
g.loc[(g['firstWalk'] - g['signuptime']) >= timedelta(days=7), ['firstWalkinWeek']] = 'NO' 

# lastMonthWalks (Number of walks done by walker in last 30 days)

#totalActiveDayCount 
#total number of walk day: number of days that walker has walking


In [184]:
# Creating lastMonthWalks
df3['checkintime'] = pd.to_datetime(df3['checkintime'])
thirty_days_ago = df3['checkintime'].max() - pd.to_timedelta("30day")
thirty_days_ago
g['lastMonthWalks'] = df3[df3.checkintime>=thirty_days_ago].groupby('walkerid').agg({'walkid': pd.Series.nunique})
r = g.copy()

In [39]:
# totalActiveDaycount
df3copy['checkintime'] = pd.to_datetime(df3copy['checkintime'])
df3copy['days'] = df3copy['checkintime'].dt.strftime('%d')
g['totalActiveDayCount'] = df3copy.groupby('walkerid').agg({'days' : pd.Series.nunique})
# totalWalk/totalActiveDayCount
g['dailyWalkAverage'] = g['totalWalks']/g['totalActiveDayCount']
g['activeLifeTime'] = g['lastWalk'] - g['firstWalk']
g['walkFrequency'] = g['activeLifeTime']/g['totalActiveDayCount']



In [185]:
# Adding lastMonthWalks to the main table
new = pd.merge(r.reset_index(), t.reset_index(), how = 'outer')
new

In [188]:
# Replacing NaN values with 0
new["lastMonthWalks"] = new["lastMonthWalks"].fillna(0)
new

,walkerid,totalWalks,firstWalk,lastWalk,signuptime,differentDistrictCount,averageWalk,lifetime,averageLifetime,firstWalkAfterSignup,firstWalkinWeek,totalActiveDayCount,dailyWalkAverage,activeLifeTime,walkFrequency,lastMonthWalks
0,01b0fb51-4461-4638-95a5-8fc8c0ef1c9d,2,2020-03-10 11:15:00.000,2020-03-12 12:00:00,2020-03-09 14:52:09.482,1,LESS,2 days 21:07:50.518000,LESS,0 days 20:22:50.518000,YES,2,1.000000,2 days 00:45:00,1 days 00:22:30,0.0
1,01d9cdcd-c770-4d29-8ca5-dd0770962b67,3,2020-02-10 15:45:00.000,2020-03-15 05:00:00,2020-02-08 17:45:59.828,1,LESS,35 days 11:14:00.172000,LESS,1 days 21:59:00.172000,YES,3,1.000000,33 days 13:15:00,11 days 04:25:00,0.0
2,01fa4741-4138-472b-bdb6-d1f19fd2f44a,1,2019-07-17 17:00:00.000,2019-07-17 17:00:00,2019-07-01 14:50:38.745,1,LESS,16 days 02:09:21.255000,LESS,16 days 02:09:21.255000,NO,1,1.000000,0 days 00:00:00,0 days 00:00:00,0.0
3,03b85e74-989f-4cf2-993f-d8411de50e3a,4,2020-02-05 10:54:30.993,2020-02-07 13:45:00,2020-02-04 09:59:30.186,2,LESS,3 days 03:45:29.814000,LESS,1 days 00:55:00.807000,YES,3,1.333333,2 days 02:50:29.007000,0 days 16:56:49.669000,0.0
4,045cc0a5-afb1-4caa-9575-7faf6d3e1f69,18,2020-03-05 12:00:00.000,2020-04-15 08:30:00,2020-03-04 12:39:17.707,2,LESS,41 days 19:50:42.293000,LESS,0 days 23:20:42.293000,YES,15,1.200000,40 days 20:30:00,2 days 17:22:00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,fc597c8e-65bc-48f6-b5d9-b372238cd94d,5,2020-03-29 18:00:00.000,2020-09-29 09:00:00,2020-03-15 17:49:50.346,2,LESS,197 days 15:10:09.654000,MORE,14 days 00:10:09.654000,NO,4,1.250000,183 days 15:00:00,45 days 21:45:00,0.0
351,fd1c862f-25a6-4051-a9b5-2776fb28345e,2,2019-08-31 17:00:00.000,2019-10-18 07:00:00,2019-08-26 09:24:47.883,1,LESS,52 days 21:35:12.117000,LESS,5 days 07:35:12.117000,YES,2,1.000000,47 days 14:00:00,23 days 19:00:00,0.0
352,fde176dc-7f6b-408f-8c3e-6835b651d5ba,23,2020-01-25 18:00:00.000,2020-08-22 15:30:00,2020-01-24 12:44:10.224,2,MORE,211 days 02:45:49.776000,MORE,1 days 05:15:49.776000,YES,19,1.210526,209 days 21:30:00,11 days 01:07:53.684210526,0.0
353,ff665506-7322-4fcf-adef-0fdc52a8bd71,171,2019-09-14 09:00:00.000,2021-02-25 15:45:00,2019-09-09 18:54:57.236,3,MORE,534 days 20:50:02.764000,MORE,4 days 14:05:02.764000,YES,31,5.516129,530 days 06:45:00,17 days 02:32:25.161290322,3.0


In [229]:
# activityTag
new.loc[(new['lastMonthWalks'] > 0), ['activityTag']] = 'YES'
new.loc[(new['lastMonthWalks'] == 0), ['activityTag']] = 'NO'
new

,walkerid,totalWalks,firstWalk,lastWalk,signuptime,differentDistrictCount,averageWalk,lifetime,averageLifetime,firstWalkAfterSignup,firstWalkinWeek,totalActiveDayCount,dailyWalkAverage,activeLifeTime,walkFrequency,lastMonthWalks,activityTag
0,01b0fb51-4461-4638-95a5-8fc8c0ef1c9d,2,2020-03-10 11:15:00.000,2020-03-12 12:00:00,2020-03-09 14:52:09.482,1,LESS,2 days 21:07:50.518000,LESS,0 days 20:22:50.518000,YES,2,1.000000,2 days 00:45:00,1 days 00:22:30,0.0,NO
1,01d9cdcd-c770-4d29-8ca5-dd0770962b67,3,2020-02-10 15:45:00.000,2020-03-15 05:00:00,2020-02-08 17:45:59.828,1,LESS,35 days 11:14:00.172000,LESS,1 days 21:59:00.172000,YES,3,1.000000,33 days 13:15:00,11 days 04:25:00,0.0,NO
2,01fa4741-4138-472b-bdb6-d1f19fd2f44a,1,2019-07-17 17:00:00.000,2019-07-17 17:00:00,2019-07-01 14:50:38.745,1,LESS,16 days 02:09:21.255000,LESS,16 days 02:09:21.255000,NO,1,1.000000,0 days 00:00:00,0 days 00:00:00,0.0,NO
3,03b85e74-989f-4cf2-993f-d8411de50e3a,4,2020-02-05 10:54:30.993,2020-02-07 13:45:00,2020-02-04 09:59:30.186,2,LESS,3 days 03:45:29.814000,LESS,1 days 00:55:00.807000,YES,3,1.333333,2 days 02:50:29.007000,0 days 16:56:49.669000,0.0,NO
4,045cc0a5-afb1-4caa-9575-7faf6d3e1f69,18,2020-03-05 12:00:00.000,2020-04-15 08:30:00,2020-03-04 12:39:17.707,2,LESS,41 days 19:50:42.293000,LESS,0 days 23:20:42.293000,YES,15,1.200000,40 days 20:30:00,2 days 17:22:00,0.0,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,fc597c8e-65bc-48f6-b5d9-b372238cd94d,5,2020-03-29 18:00:00.000,2020-09-29 09:00:00,2020-03-15 17:49:50.346,2,LESS,197 days 15:10:09.654000,MORE,14 days 00:10:09.654000,NO,4,1.250000,183 days 15:00:00,45 days 21:45:00,0.0,NO
351,fd1c862f-25a6-4051-a9b5-2776fb28345e,2,2019-08-31 17:00:00.000,2019-10-18 07:00:00,2019-08-26 09:24:47.883,1,LESS,52 days 21:35:12.117000,LESS,5 days 07:35:12.117000,YES,2,1.000000,47 days 14:00:00,23 days 19:00:00,0.0,NO
352,fde176dc-7f6b-408f-8c3e-6835b651d5ba,23,2020-01-25 18:00:00.000,2020-08-22 15:30:00,2020-01-24 12:44:10.224,2,MORE,211 days 02:45:49.776000,MORE,1 days 05:15:49.776000,YES,19,1.210526,209 days 21:30:00,11 days 01:07:53.684210526,0.0,NO
353,ff665506-7322-4fcf-adef-0fdc52a8bd71,171,2019-09-14 09:00:00.000,2021-02-25 15:45:00,2019-09-09 18:54:57.236,3,MORE,534 days 20:50:02.764000,MORE,4 days 14:05:02.764000,YES,31,5.516129,530 days 06:45:00,17 days 02:32:25.161290322,3.0,YES


In [222]:
# didDemandOrWalk
# I did not want to append the walkers without any walks to the original table

# merging with (how = 'right') to get walkers both with or without walks 
lastdf = pd.merge(df1,df2, how = 'right', left_on = 'walkerid', right_on = 'id')
lastdf
last2df = lastdf.groupby('id').agg({'walkid' : ['count']})
last2df.columns = ['walks']
last2df
last2df.loc[last2df['walks'] > 0, ['didDemandOrWalk']] = 'YES'
last2df.loc[last2df['walks'] == 0, ['didDemandOrWalk']] = 'NO'
last2df

,walks,didDemandOrWalk
id,,
004936aa-fb2c-402f-9738-676ca131575a,0,NO
00e99cd8-9886-4808-8266-7080bcf2ac88,0,NO
01029542-74c8-4ba7-9370-c63d0eab244d,0,NO
01b0fb51-4461-4638-95a5-8fc8c0ef1c9d,2,YES
01d9cdcd-c770-4d29-8ca5-dd0770962b67,3,YES
...,...,...
fdcd70a8-889a-4205-87df-6047bc4e0404,0,NO
fde176dc-7f6b-408f-8c3e-6835b651d5ba,23,YES
fe99109f-6d1e-48d7-aad4-b9c4198cc0ab,0,NO
